# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df= pd.read_csv("cities.csv")
cities_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,3833367,Ushuaia,AR,-54.8000,-68.3000,49.66,71,75,0.00
1,2015852,Svetlaya,RU,46.5389,138.3297,52.16,69,1,8.99
2,1651810,Airai,TL,-8.9266,125.4092,60.91,91,100,2.80
3,4030556,Rikitea,PF,-23.1203,-134.9692,73.87,66,0,14.79
4,2088659,Panguna,PG,-6.3164,155.4848,68.97,95,64,1.97
...,...,...,...,...,...,...,...,...,...
564,352628,Maţāy,EG,28.4190,30.7792,95.11,15,0,19.48
565,3174050,Province of Mantua,IT,45.1667,10.7833,75.58,34,66,3.00
566,1791779,Wanning,CN,18.8003,110.3967,81.84,89,49,9.89
567,1307741,Myitkyina,MM,25.3833,97.4000,78.76,65,96,5.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
Perfect_df = cities_df[(cities_df['Temperature'] >= 75) & (cities_df['Temperature'] <= 90)]
Perfect_df = Perfect_df[Perfect_df['Wind Speed'] <= 10]
Perfect_df = Perfect_df[Perfect_df['Clouds'] <= 10]
Perfect_df = Perfect_df[Perfect_df['Humidity'] <= 70]
Hotel_df = Perfect_df
Hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
48,932987,Tsabong,BW,-26.0217,22.4010,80.04,18,0,9.42
59,964432,Port Alfred,ZA,-33.5906,26.8910,75.43,39,7,2.53
73,2264557,Ponta do Sol,PT,32.6667,-17.1000,75.54,53,1,4.88
101,1282256,Hithadhoo,MV,-0.6000,73.0833,84.81,69,0,4.90
219,3465342,Corumbá,BR,-19.0092,-57.6533,76.91,69,0,2.30
231,3351663,Benguela,AO,-12.5763,13.4055,85.44,58,1,8.30
257,937136,Zeerust,ZA,-25.5369,26.0751,76.01,14,8,6.91
275,3351024,Caluquembe,AO,-13.7833,14.6833,77.32,30,5,4.43
277,1513087,Nurota,UZ,40.5614,65.6886,89.69,11,0,7.09
338,301975,Samandağ,TR,36.0842,35.9771,81.12,59,0,7.85


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
Hotel_df['Hotel Name'] = ""
Hotel_df

for index, row in Hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        Hotel_data = requests.get(base_url, params=params).json()
        
        Hotel_df.loc[index, "Hotel Name"] = Hotel_data["results"][0]["name"]
        
    except IndexError:
        
        Hotel_df.loc[index, "Hotel Name"] = "NaN"
        
Hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
48,932987,Tsabong,BW,-26.0217,22.4010,80.04,18,0,9.42,Batshweneng Guest House
59,964432,Port Alfred,ZA,-33.5906,26.8910,75.43,39,7,2.53,Oceana Beach Wildlife Reserve
73,2264557,Ponta do Sol,PT,32.6667,-17.1000,75.54,53,1,4.88,Estalagem Da Ponta Do Sol
101,1282256,Hithadhoo,MV,-0.6000,73.0833,84.81,69,0,4.90,Abuharee Grand
219,3465342,Corumbá,BR,-19.0092,-57.6533,76.91,69,0,2.30,Candeias Hotel Gold Fish
231,3351663,Benguela,AO,-12.5763,13.4055,85.44,58,1,8.30,Aparthotel Mil Cidades
257,937136,Zeerust,ZA,-25.5369,26.0751,76.01,14,8,6.91,Zeerust Accommodation - Tapologo Lodge
275,3351024,Caluquembe,AO,-13.7833,14.6833,77.32,30,5,4.43,Hotel Kalukembe
277,1513087,Nurota,UZ,40.5614,65.6886,89.69,11,0,7.09,NaN
338,301975,Samandağ,TR,36.0842,35.9771,81.12,59,0,7.85,Truva Life Hotel


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in Hotel_df.iterrows()]
locations = Hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))